In [75]:
import mido

In [68]:
def get_midi_obj(midi_file: str):
    return mido.MidiFile(midi_file, clip=True)

In [69]:
midi_file = 'midis/TLS-0.mid'
mid = get_midi_obj(midi_file)

In [70]:
len(mid.tracks) # Meta and Measure tracks

2

### Parse time signature and tempo events:
Time signatures and tempos can affect how measures are calculated. You need to extract these from the MIDI file.

In [61]:
def get_time_signatures(midi_obj) -> list:
    time_signatures = []
    for track in midi_obj.tracks:
        for msg in track:
            if msg.type == 'time_signature':
                time_signatures.append(msg)
    
    return time_signatures
                
                
def get_tempos(midi_obj) -> list:
    tempos = []
    for track in midi_obj.tracks:
        for msg in track:
            if msg.type == 'set_tempo':
                tempos.append(msg)
    
    return tempos

In [73]:
time_signatures = get_time_signatures(mid)
tempos = get_tempos(mid)
time_signatures, tempos

([MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0)],
 [MetaMessage('set_tempo', tempo=631577, time=0)])

### Calculate the length of a measure in ticks:
The length of a measure depends on the time signature and tempo. For simplicity, let's assume a constant time signature and tempo throughout the file. (Handling changes would require a more complex approach).

In [74]:
def get_measures(midi_obj) -> list:
    time_signatures = get_time_signatures(midi_obj)
    tempos = get_tempos(midi_obj)
    
    if time_signatures:
        time_signature = time_signatures[0]
        numerator = time_signature.numerator
        denominator = time_signature.denominator
    else:
        numerator = 4  # default to 4/4 time
        denominator = 4

    if tempos:
        tempo = tempos[0].tempo
    else:
        tempo = 500000  # default to 120 BPM

    ticks_per_beat = midi_obj.ticks_per_beat
    ticks_per_measure = (ticks_per_beat * 4 * numerator) / denominator
    
    current_ticks = 0
    current_measure = []
    measures = []

    for track in mid.tracks:
        for msg in track:
            current_ticks += msg.time
            current_measure.append(msg)
            if current_ticks >= ticks_per_measure:
                measures.append(current_measure)
                current_measure = []
                current_ticks = 0

    # Handle the last measure if it contains any messages
    if current_measure:
        measures.append(current_measure)
        
    return measures

In [66]:
measures = get_measures(mid)
measures

### Separate events into measures:
You can then separate the events into measures based on the calculated ticks per measure.

In [39]:
measure0 = [m for m in measures[0] if m.type=='note_on']
measure0

[Message('note_on', channel=0, note=60, velocity=77, time=0),
 Message('note_on', channel=0, note=52, velocity=63, time=0),
 Message('note_on', channel=0, note=48, velocity=64, time=0),
 Message('note_on', channel=0, note=60, velocity=83, time=12),
 Message('note_on', channel=0, note=67, velocity=90, time=12),
 Message('note_on', channel=0, note=60, velocity=77, time=0),
 Message('note_on', channel=0, note=48, velocity=71, time=0),
 Message('note_on', channel=0, note=67, velocity=87, time=12),
 Message('note_on', channel=0, note=69, velocity=86, time=12)]

In [40]:
measure0_time = [t.time for t in measure0]
measure0_time

[0, 0, 0, 12, 12, 0, 0, 12, 12]

### Save each measure as a separate MIDI file:
Finally, you can save each measure into a separate MIDI file.

In [ ]:
for i, measure in enumerate(measures):
    new_mid = mido.MidiFile()
    new_track = mido.MidiTrack()
    new_mid.tracks.append(new_track)
    new_track.extend(measure)
    new_mid.save(f'measure_{i + 1}.mid')